In [1]:
#import library
!pip install kaggle
import kaggle

In [54]:
#download the dataset from kaggle
!kaggle datasets download -d ankitbansal06/retail-orders --force

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|##########| 200k/200k [00:00<00:00, 102MB/s]


In [5]:
#extract the csv file from the zip folder 
import zipfile
zip_ref = zipfile.ZipFile('retail-orders.zip')
zip_ref.extractall()
zip_ref.close()

In [10]:
#read data from file and handle null values
import pandas as pd
df = pd.read_csv('orders.csv', na_values=['Not Available', 'unknown'])
df.head(20)
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [26]:
#rename columns..replace spaces with underscore and everything lowercase
#df.rename(columns={'Order Id':'order_id', 'City':'city'})
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')

In [25]:
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [34]:
#derive new columns.. discount, sale price and profit
df['discount'] = df['list_price']*df['discount_percent']*.01
df['sale_price'] = df['list_price'] - df['discount']
df['profit'] = df['sale_price'] - df['cost_price']

In [37]:
#convert order_date from object datatype to datetime
df['order_date'] = pd.to_datetime(df['order_date'], format="%Y-%m-%d")

In [39]:
#drop list price, cost price and discount_percent columns
df.drop(columns=['list_price','cost_price','discount_percent'], inplace=True)

In [51]:
#connect with sql server
import sqlalchemy as sal
engine = sal.create_engine('mssql://LAPTOP-QO79K6NV\SQLEXPRESS/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn = engine.connect()

In [ ]:
#load the data into sql server using replace option
#df.to_sql('df_orders', con = conn, index = False, if_exists = 'replace')

In [53]:
#load the data into sql server using append option
df.to_sql('df_orders', con = conn, index = False, if_exists = 'append')

38